In [45]:
import pandas as pd
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import random
import os
import torch.nn as nn
from torch.nn.functional import softmax


# custom dataset class for our image data
class UCMercedDataset(Dataset):
    def __init__(self,class_list, root_dir, transforms=None):
        dataX=[]
        for i in class_list:
            for j in range(100):
                dataX.append(f'{i}{j:02}.tif')
        dataX=np.array(dataX)
        np.random.shuffle(dataX)
        
        self.dataX=dataX
        self.root_dir=root_dir
        self.transforms=transforms
        self.class_list={class_list[i]:i for i in range(len(class_list))}
    
    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self,index):
        imgName=self.dataX[index]
        imgPath=os.path.join(self.root_dir, imgName[:-6], imgName)
        image=Image.open(imgPath)
        y=torch.tensor(self.class_list[imgName[:-6]])
        if self.transforms:
            image=self.transforms(image)
        return(image,y)


def train(ep=5):
    # Training loop
    for epoch in range(ep):
        for i, (img,target) in enumerate(train_loader):
            img=img.to(device)
            target=target.to(device)

            out=model(img)
            loss=lossFunction(out,target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if(i%10==0):
                print(f'\033[30m {loss.item()}')
        print(f' \033[93m epoch {epoch+1} Finished\n\n')


# This is a helper function used only once to calculate the mean and std for the training data
def get_mean_std(loader):
    mean=0
    std=0
    total_image=0
    for i, _ in loader:
        batch_size=i.size(0)
        imgs=i.view(batch_size,i.size(1),-1)
        mean+=imgs.mean(2).sum(0)
        std+=imgs.std(2).sum(0)
        total_images=batch_size
    
    mean/=total_images
    std/=total_images
    return mean, std

# This is a helper function used to calculate the accuracy of the passed loader
def accuracy(loader, verbose=False):
    correct=0
    with torch.no_grad():
        for i, (img,target) in enumerate(loader):
            img=img.to(device)
            target=target.to(device)
            out=model(img)
            out=softmax(out,dim=1)
            for j in range(len(out)):
                if torch.argmax(out[j]) == target[j]:
                    if(verbose):
                        print(f'\033[92m target:{target[j]} prediction:{torch.argmax(out[j])} confidence:{round(torch.max(out[j]).item()*100,2)}%')
                    correct+=1
                else:
                    if(verbose):
                        print(f'\033[91m target: {target[j]} prediction: {torch.argmax(out[j])} confidence: {round(torch.max(out[j]).item()*100,2)}%')

    print(f'\033[94m Accuracy: {correct*100/(len(loader)*batch_size)}')


batch_size=32

# calculated mean and std of the entire data calculated using get_mean_std() helper function
mean=torch.tensor([62.4188, 62.6786, 62.6386])
std=torch.tensor([23.9611, 22.5873, 21.3868])

# Defining a custom transform pipeline
train_transform=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomInvert(0.6),
    transforms.RandomSolarize(0.6),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,std=std),
#     transforms.ColorJitter(),
    
])

#Getting GPU support if available
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    



classes=os.listdir("/kaggle/input/ucmerced/UCMerced_LandUse/Images")
# randomly selecting 14 classes for training
classes_for_training=random.sample(classes, 14)
classes_left_out=list(set(classes)-set(classes_for_training))



data=UCMercedDataset(class_list=classes_for_training,
                               root_dir="/kaggle/input/ucmerced/UCMerced_LandUse/Images",
                               transforms=train_transform)

training_data, testing_data =random_split(data, [1000,400])

remaining_data=UCMercedDataset(class_list=classes_left_out,
                               root_dir="/kaggle/input/ucmerced/UCMerced_LandUse/Images",
                               transforms=train_transform)


# Dataloader for training dataset
train_loader=DataLoader(dataset=training_data, batch_size=batch_size,shuffle=True)
# Dataloader for testing dataset
test_loader=DataLoader(dataset=testing_data, batch_size=batch_size,shuffle=True)
# data loader for the untrainied classes
remaining_loader=DataLoader(dataset=remaining_data, batch_size=batch_size)




# loading the pretrained model for the resnet
model=torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
num_fters=model.fc.in_features
# setting the last layer of the model to give output for the 14 classes 
model.fc=nn.Linear(num_fters, 14)
model=model.to(device)
# using cross entropy loss function
lossFunction=nn.CrossEntropyLoss()
# using adam optimizer
optimizer=torch.optim.Adam(model.parameters(), lr=0.0001)    


print("Training...")
train(ep=10)

# Training accuracy
accuracy(train_loader)
print(f"\033[30m Training accuracy\n\n")

# Test accuracy
accuracy(test_loader, verbose=True)
print(f"\033[30m Test accuracy\n\n")

# Accuracy for the classes which were not even in the training
accuracy(remaining_loader, verbose=True)
print("Accuracy on the classes which were not even used in the training process\n\n")

# 995
# 99.5
# 384
# 96.0

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Training...
 2.825975179672241
 1.632214903831482
 0.9529239535331726
 0.8187040686607361
  epoch 1 Finished


 0.6329565644264221
 0.29506510496139526
 0.38541123270988464
 0.29911303520202637
  epoch 2 Finished


 0.11797478049993515
 0.2875511348247528
 0.22518481314182281
 0.12293360382318497
  epoch 3 Finished


 0.03468773141503334
 0.06378623098134995
 0.08401776850223541
 0.09371339529752731
  epoch 4 Finished


 0.07813192158937454
 0.033857788890600204
 0.0496072955429554
 0.18473246693611145
  epoch 5 Finished


 0.030771616846323013
 0.0761777013540268
 0.033028095960617065
 0.07644280046224594
  epoch 6 Finished


 0.016225477680563927
 0.018037943169474602
 0.013957143761217594
 0.008556254208087921
  epoch 7 Finished


 0.029684484004974365
 0.03706371411681175
 0.034119877964258194
 0.21516315639019012
  epoch 8 Finished


 0.018442748114466667
 0.035950858145952225
 0.009289397858083248
 0.009316559880971909
  epoch 9 Finished


 0.0034640568774193525
 0.04090943560004

tensor([10.2500])